In [3]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

2022-11-30 15:47:03.184200: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [1]:
def create_model(input, n_vocab):
    model = Sequential()

    model.add(LSTM(
        units=512,
        input_shape = (input.shape[1], input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))

    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3))

    model.add(LSTM(256))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))

    #if using weights-improvement-188-0.0081-bigger.hdf5 weights, comment out the following lines
    #if using weights-improvement-100.hdf5 weights, uncomment the following lines
    # model.add(Dense(n_vocab))
    # model.add(Dropout(0.3))
    # model.add(Dense(n_vocab))
    # model.add(Dropout(0.3))
    #endif

    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

# Train model

In [4]:
import pickle
with open('../data/network_input','rb') as file:
    network_input = pickle.load(file)
with open('../data/network_output','rb') as file:
    network_output = pickle.load(file)
with open('../data/notes','rb') as file:
    notes = pickle.load(file)

num_vocab = len(set(notes))

filepath = "weights-improvement-{epoch:02d}.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]
model = create_model(network_input,num_vocab)
model.fit(network_input, network_output, epochs=100, batch_size=64, callbacks=callbacks_list)

2022-11-30 15:47:19.094821: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-30 15:47:19.096510: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/100
11/11 [==============================] - 44s 3s/step - loss: 4.7640
Epoch 2/100
11/11 [==============================] - 32s 3s/step - loss: 4.8242
Epoch 3/100
11/11 [==============================] - 34s 3s/step - loss: 4.6474
Epoch 4/100
11/11 [==============================] - 32s 3s/step - loss: 4.6178
Epoch 5/100
11/11 [==============================] - 31s 3s/step - loss: 4.5835
Epoch 6/100
11/11 [==============================] - 34s 3s/step - loss: 4.6420
Epoch 7/100
11/11 [==============================] - 34s 3s/step - loss: 4.5665
Epoch 8/100
11/11 [==============================] - 35s 3s/step - loss: 4.5648
Epoch 9/100
11/11 [==============================] - 34s 3s/step - loss: 4.5342
Epoch 10/100
11/11 [==============================] - 34s 3s/step - loss: 4.5415
Epoch 11/100
11/11 [==============================] - 34s 3s/step - loss: 4.5285
Epoch 12/100
11/11 [==============================] - 34s 3s/step - loss: 4.5184
Epoch 13/100
11/11 [=================

In [5]:
import pickle

with open('../data/network_input','rb') as file:
    network_input = pickle.load(file)
with open('../data/network_output','rb') as file:
    network_output = pickle.load(file)
with open('../data/notes','rb') as file:
    notes = pickle.load(file)
with open('../data/note_dict','rb') as file:
    note_dict = pickle.load(file)
with open('../data/pitch_names','rb') as file:
    pitch_names = pickle.load(file)


model = create_model(network_input,num_vocab)
model.load_weights("weights-improvement-188-0.0081-bigger.hdf5")
# model.load_weights("weights-improvement-100.hdf5")


startIndex = np.random.randint(0,len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitch_names))
pattern = network_input[startIndex]
prediction_output = []

for i in range(500):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(num_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern = np.append(pattern,index)
    pattern = pattern[1:len(pattern)]

offset = 0
output_notes = []
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output.mid')

'test_output.mid'